In [213]:
import requests
import json
import pandas as pd
import csv
import os

from datetime import datetime

In [214]:
# Retrieves useful date information

now = datetime.now()

current_date = now.strftime("%d-%B-%Y")
current_month_int = now.strftime("%m")

print("Current Time is :", current_date)

current_month = now.strftime("%B")
current_year = current_date[-4:]

current_date = now.strftime("%d-%m-%Y")

Current Time is : 29-April-2022


In [215]:
# # Si lo tuvieramos que hacer en español usariamos este diccionario

# calendar = {'01':'Enero',
#             '02':'Febrero',
#             '03':'Marzo',
#             '04':'Abril',
#             '05':'Mayo',
#             '06':'Junio',
#             '07':'Julio',
#             '08':'Agosto',
#             '09':'Septiembre',
#             '10':'Octubre',
#             '11':'Noviembre',
#             '12':'Diciembre',
# }

# now = datetime.now()

# current_time = now.strftime("%d-%m-%Y")
# current_month_int = now.strftime("%m")

# print("Current Time is :", current_time)

# current_month = calendar[current_month_int]
# current_year = current_time[-4:]

In [216]:
# Creates the basic folder structure

def createFolderStructure(datasetName):
    root = 'datasets/' + datasetName
    directory = current_year + "-" + current_month

    path = os.path.join(root,directory)
    if not os.path.exists(path):
        os.makedirs(path)

In [217]:
# Pide los datasets desde las webs y los parsea en un csv
# Aclaración el regex es por que hay ciertos datos que estan mal formateados ejemplo "Juan , Pablo" , esto debería estar en la misma columna.
# Pero en un CSV una coma (o el separador que se utilize) significa una nueva columna.
# Utilizando el regex identificamos estos casos molestos, la desventaja es que es mas lento la ventaja es que ganamos 141+4+27 filas mas
 

import re
def retrieve_and_save(datasetSource, datasetName):
    
    createFolderStructure(datasetName)

    r = requests.get(datasetSource)
    if r.status_code!=200:
            print ("Failure!!")
            exit()
    
    path = os.path.join('datasets',datasetName,current_year + "-" + current_month)

    with open(path + '/' + datasetName + '-' + current_date + '.csv', 'w') as f:
        writer = csv.writer(f)
        contador= 0
        for line in r.iter_lines():
            decoded_line = line.decode('utf-8')
            

            if '"' in decoded_line:
                pattern = r'\".*,.*\"' 
                x = re.findall(pattern, line.decode('utf-8'))
                
                if len(x) != 0:
                    replacement = x[0].replace(',',' ')
                    # print(replacement)
                    
                    decoded_line = decoded_line.replace(x[0],replacement)
                contador += 1
                # print('problemas')

            if "'" in decoded_line:
                decoded_line = decoded_line.replace("'","''")

            if '’' in decoded_line:
                decoded_line = decoded_line.replace("’","''")
            if '`' in decoded_line:
                decoded_line = decoded_line.replace("`","''")

                #print(decoded_line)
            if '"' in decoded_line:
                decoded_line = decoded_line.replace('"',"''")
                # print(decoded_line)
                
            if 's/d' in decoded_line:
                decoded_line = decoded_line.replace('s/d','NULL')

            if ',,' in decoded_line:
                print(decoded_line)
                decoded_line = decoded_line.replace(',,',',NULL,')
                print(decoded_line)
            
            
            writer.writerow(decoded_line.split(','))
        print(contador)


In [218]:
def dataGatherer():
    web1 = "https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/4207def0-2ff7-41d5-9095-d42ae8207a5d/download/museos_datosabiertos.csv"
    web2 = "https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/392ce1a8-ef11-4776-b280-6f1c7fae16ae/download/cine.csv"
    web3 = "https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/01c6c048-dbeb-44e0-8efa-6944f73715d7/download/biblioteca_popular.csv"

    retrieve_and_save (web1,'museos')
    retrieve_and_save (web2,'salas-de-cine')
    retrieve_and_save (web3,'bibliotecas-populares')
    
    print("Data gathered succesfully")

In [219]:
dataGatherer()

06588100,06,06588,,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,9 de Julio,Archivo Y Museo Histórico Gral. Julio De Vedia,Libertad 1191,,B6500EVL,2317,425 279,archivoymuseo@yahoo.com.ar,www.portaldel9.com.ar,-35.4417620,-60.8875980,Localización precisa,,DNPyM,Municipal,1920,2017
06588100,06,06588,NULL,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,9 de Julio,Archivo Y Museo Histórico Gral. Julio De Vedia,Libertad 1191,NULL,B6500EVL,2317,425 279,archivoymuseo@yahoo.com.ar,www.portaldel9.com.ar,-35.4417620,-60.8875980,Localización precisa,NULL,DNPyM,Municipal,1920,2017
06077010,06,06077,,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,Arrecifes,Museo Y Archivo Histórico De Arrecifes,Gerardo Risso y España,,B2740FMJ,2478,452 931,museoarrecifes@hotmail.com,,-34.0669700,-60.1025550,Localización precisa,,DNPyM,Municipal,1972,2017
06077010,06,06077,NULL,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,Arrecifes,Museo Y Archivo Histórico De Arrecifes,Gerardo Riss